In [111]:
!pip install pandas numpy matplotlib seaborn missingno plotly openpyxl pyarrow fastparquet

# load all packages for EDA
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import plotly.express as px

# load the data
data = pd.read_excel("../data/CDS_25_Task1.xlsx", sheet_name='Data', header=1, index_col=False)


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### Emotional, workplace, and environmental factor codes
| Group | Subgroup | Statement | Fast Choice Keys | Likert Keys |
|-------|----------|-----------|------------------|-------------|
| Emotional | General | I'm ready for my next step | Fgen | Lgen |
| Emotional | Transformation | I want a new start | FTraDes | LTraDes |
| Emotional | Transformation | I've got freedom to change | FTraAut | LTraAut |
| Emotional | Containers | I'm comfortable where I am | FCntCom | LCntCom |
| Emotional | Containers | I feel like my voice is heard | FCntPsy | LCntPsy |
| Emotional | Connection | Others influence my decisions | FConSoc | LConSoc |
| Emotional | Balance | I'm anxious about change | FBalAnx | LBalAnx |
| Emotional | Resources | I'm financially motivated | FResFin | LResFin |
| Emotional | Control | I believe in myself | FContImp | LContImp |
| Emotional | Journey | I'm optimistic about the future | FJouOpt | LJouOpt |
| Emotional | Connection | I feel included | FConInc | LConInc |
| Emotional | Balance | I'm happy where I am | FBalSat | LBalSat |
| Emotional | Resources | I've got the skills to progress | FResSki | LResSki |
| Emotional | Control | I control my next step | FContCon | LContCon |
| Emotional | Journey | I've set myself goals | FJouPro | LJouPro |
| Workplace/functional | Lack of opportunity to use skills/abilities | I can use my skills | FUseSkills | LUseSkills |
| Workplace/functional | Learning and development | I have opportunities to learn | FLearnDev | LLearnDev |
| Workplace/functional | Career advancement and promotions | I can grow here | Fcarprom | Lcarprom |
| Workplace/functional | Meaning | I find my job meaningful | FMeanFull | LMeanFull |
| Workplace/functional | Poor management | My manager is poor | Fpoorman | Lpoorman |
| Workplace/functional | Toxic workplace/Company culture | The work culture is toxic | Ftoxic | Ltoxic |
| Workplace/functional | Excessive or too little work | I'm working too hard | FExcessWk | LExcessWk |
| Workplace/functional | Disagreement or fall out with colleagues | I don't get along with my colleagues | Fcollea | Lcollea |
| Workplace/functional | A better salary and financial stability | I'm well compensated | Fwellcomp | Lwellcomp |
| Workplace/functional | Financial fairness | My salary is unfair compared to colleagues | FFinFair | LFinLair |
| Workplace/functional | Satisfaction around hybrid working | I enjoy hybrid working | Fenjhyb | Lenjhyb |

### Data Cleaning

Firstly, clean our variable names, set our data types, and re-save the data for future loading/processing.

In [112]:

# Read the data
data = pd.read_excel("../data/CDS_25_Task1.xlsx", sheet_name='Data', 
                    header=1, index_col=False)

# Quick rename with category prefixes
cats = pd.read_excel("../data/CDS_25_Task1.xlsx", sheet_name='Data', 
                    nrows=1, header=None).iloc[0].fillna(method='ffill')

data.columns = [f"{'pers' if 'personality' in str(cat).lower() else str(cat)[:4]}_{col}".lower().replace(' ', '_') 
                for cat, col in zip(cats, data.columns)]

data.head()

print(f"Data shape: {data.shape}")

print(f"Data columns: {data.columns}")

Data shape: (4211, 67)
Data columns: Index(['batc_unnamed:_0', 'gend_unnamed:_1', 'age_unnamed:_2',
       'pers_extraverted,_enthusiastic.', 'pers_critical,_quarrelsome.',
       'pers_dependable,_self-disciplined.', 'pers_anxious,_easily_upset.',
       'pers_open_to_new_experiences,_complex.', 'pers_sympathetic,_warm.',
       'pers_disorganized,_careless.', 'pers_reserved,_quiet.',
       'pers_calm,_emotionally_stable.', 'pers_conventional,_uncreative.',
       'prep_unnamed:_13', 'qual_unnamed:_14',
       'emot_emotional_likert.lbal_anx_', 'emot_emotional_likert.lbal_sat_',
       'emot_emotional_likert.lcnt_com_', 'emot_emotional_likert.lcnt_psy_',
       'emot_emotional_likert.lcon_inc_', 'emot_emotional_likert.lcon_soc_',
       'emot_emotional_likert.lcont_con__',
       'emot_emotional_likert.lcont_imp__', 'emot_emotional_likert.lgen__',
       'emot_emotional_likert.ljou_opt__', 'emot_emotional_likert.ljou_pro__',
       'emot_emotional_likert.lres_fin__', 'emot_emotional_

C:\Users\rhrou\AppData\Local\Temp\ipykernel_56424\3271360133.py:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  nrows=1, header=None).iloc[0].fillna(method='ffill')


In [113]:
# clean column names
def clean_column_names(df):
    """Clean dataframe column names using simple string operations"""
    
    # Create mapping dictionary
    name_mapping = {}
    
    for col in df.columns:
        new_name = col
        
        # Handle specific replacements first
        if col == 'batc_unnamed:_0':
            new_name = 'batch'
        elif col == 'gend_unnamed:_1':
            new_name = 'gender'
        elif col == 'age_unnamed:_2':
            new_name = 'age'
        elif col == 'prep_unnamed:_13':
            new_name = 'prep_level'
        elif col == 'qual_unnamed:_14':
            new_name = 'qual_reasons'
        else:
            # Replace long prefixes with short codes
            new_name = new_name.replace('emot_emotional_statements', 'es') # change to 'es' emotional statements (fast choice response)
            new_name = new_name.replace('emot_emotional_likert', 'el') # change to workplace/functional likert, wfl
            new_name = new_name.replace('work_functional_statements', 'wfs') #change workplace/functional statements (fast choice response)
            new_name = new_name.replace('work_functional_likert', 'wfl') # change workplace/functional likert, wfl
            
            # Replace periods with underscores
            new_name = new_name.replace('.', '_')
            
            # Remove commas
            new_name = new_name.replace(',', '')
            
            # Remove trailing underscores
            new_name = new_name.rstrip('_')
        
        name_mapping[col] = new_name
    
    # Apply the mapping
    df_cleaned = df.rename(columns=name_mapping)
    
    return df_cleaned

data = clean_column_names(data)

data.columns

Index(['batch', 'gender', 'age', 'pers_extraverted_enthusiastic',
       'pers_critical_quarrelsome', 'pers_dependable_self-disciplined',
       'pers_anxious_easily_upset', 'pers_open_to_new_experiences_complex',
       'pers_sympathetic_warm', 'pers_disorganized_careless',
       'pers_reserved_quiet', 'pers_calm_emotionally_stable',
       'pers_conventional_uncreative', 'prep_level', 'qual_reasons',
       'el_lbal_anx', 'el_lbal_sat', 'el_lcnt_com', 'el_lcnt_psy',
       'el_lcon_inc', 'el_lcon_soc', 'el_lcont_con', 'el_lcont_imp', 'el_lgen',
       'el_ljou_opt', 'el_ljou_pro', 'el_lres_fin', 'el_lres_ski',
       'el_ltra_aut', 'el_ltra_des', 'wfl_lcarprom_d', 'wfl_lcollea',
       'wfl_lenjhyb', 'wfl_lexcess_wk', 'wfl_lfin_lair', 'wfl_llearn_dev',
       'wfl_lmean_full', 'wfl_lpoorman', 'wfl_ltoxic', 'wfl_luse_skills',
       'wfl_lwellcomp', 'es_fbal_anx', 'es_fbal_sat', 'es_fcnt_com',
       'es_fcnt_psy', 'es_fcon_inc', 'es_fcon_soc', 'es_fcont_con',
       'es_fcont_imp', 

In [114]:
# Exploring our variable ranges and unique values

# Categorical variables
print("=== CATEGORICAL VARIABLES ===")
for col in ['batch', 'gender', 'age', 'qual_reasons']:
    if col in data.columns:
        print(f"{col}: {data[col].nunique()} unique values")
        print(f"Values: {data[col].unique()}")
        print()

# Personality variables (pers_)
print("=== PERSONALITY VARIABLES ===")
pers_cols = [col for col in data.columns if col.startswith('pers_')]
for col in pers_cols:
    unique_vals = sorted(data[col].unique())
    print(f"{col}: {min(unique_vals)}-{max(unique_vals)}, values: {unique_vals}")

# Likert variables (eml_, wfl_)
print("\n=== LIKERT VARIABLES ===")
likert_cols = [col for col in data.columns if col.startswith(('el_', 'wfl_'))]
for col in likert_cols:
    unique_vals = sorted(data[col].dropna().unique())
    print(f"{col}: {min(unique_vals)}-{max(unique_vals)}, values: {unique_vals}")

# Factor variables (emt_, wfs_)
print("\n=== FACTOR VARIABLES ===")
factor_cols = [col for col in data.columns if col.startswith(('es_', 'wfs_'))]
for col in factor_cols:
    non_null = data[col].dropna()
    if len(non_null) > 0:
        print(f"{col}: range {non_null.min():.3f} to {non_null.max():.3f}, {non_null.nunique()} unique values")

# Prep level
print("\n=== PREP LEVEL ===")
prep_vals = sorted(data['prep_level'].dropna().unique())
print(f"prep_level: {min(prep_vals)} to {max(prep_vals)}")
print(f"Values: {prep_vals}")
print(data['prep_level'].value_counts().sort_index())


=== CATEGORICAL VARIABLES ===
batch: 7 unique values
Values: ['FR Tech' 'DE Fin' 'SP Fin' 'US Pharma' 'IT Fin' 'UK Energy' 'US Tech']

gender: 5 unique values
Values: ['Female' 'Male' 'Non-Binary / Non-Conforming' 'Prefer Not to Answer'
 'Other']

age: 3 unique values
Values: ['18-24' '25-40' '41-64']

qual_reasons: 4107 unique values
Values: ['Je suis prête à m’engager, organise'
 'Jeune diplôme je suis prête à entrer dans le monde professionnel sur un emploi stable et sur du long terme'
 'La nouvelle technologie, les nouveaux moyens mit en place…' ...
 'Italian reasons' 'El Reasons' 'Reasons Spanish']

=== PERSONALITY VARIABLES ===
pers_extraverted_enthusiastic: 1-7, values: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7)]
pers_critical_quarrelsome: 1-7, values: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7)]
pers_dependable_self-disciplined: 1-7, values: [np.int64(1), np.int64(2), np.int64(3), np.

In [115]:
# From the above, we can appropriately set our data types
# Convert categorical variables to category dtype
data['batch'] = data['batch'].astype('category')
data['gender'] = data['gender'].astype('category') 
data['qual_reasons'] = data['qual_reasons'].astype('object') # Keep qualitative as a string

# Age as ordered categorical
age_dtype = pd.CategoricalDtype(categories=['18-24', '25-40', '41-64'], ordered=True)
data['age'] = data['age'].astype(age_dtype)


# Personality variables (1-7 scale) - ordered categorical, great for ordinal survey responses
pers_dtype = pd.CategoricalDtype(categories=[1,2,3,4,5,6,7], ordered=True)
pers_cols = [col for col in data.columns if col.startswith('pers_')]
for col in pers_cols:
    data[col] = data[col].astype(pers_dtype)

# EML Likert variables (-100 to 100, steps of 25) - ordered categorical  
eml_dtype = pd.CategoricalDtype(categories=[-100,-75,-50,-25,0,25,50,75,100], ordered=True)
eml_cols = [col for col in data.columns if col.startswith('eml_')]
for col in eml_cols:
    data[col] = data[col].astype(eml_dtype)

# WFL Likert variables (0 to 100, steps of 25) - ordered categorical
wfl_dtype = pd.CategoricalDtype(categories=[0,25,50,75,100], ordered=True)
wfl_cols = [col for col in data.columns if col.startswith('wfl_')]
for col in wfl_cols:
    data[col] = data[col].astype(wfl_dtype)

# Factor variables - keep as float32 (continuous scores, need decimal precision)
emt_cols = [col for col in data.columns if col.startswith('emt_')]
for col in emt_cols:
    data[col] = data[col].astype('float32')

wfs_cols = [col for col in data.columns if col.startswith('wfs_')]
for col in wfs_cols:
    data[col] = data[col].astype('float32')

# Prep level - keep as float32 (has decimal values)
data['prep_level'] = data['prep_level'].astype('float32')

In [116]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4211 entries, 0 to 4210
Data columns (total 67 columns):
 #   Column                                Non-Null Count  Dtype   
---  ------                                --------------  -----   
 0   batch                                 4211 non-null   category
 1   gender                                4211 non-null   category
 2   age                                   4211 non-null   category
 3   pers_extraverted_enthusiastic         4211 non-null   category
 4   pers_critical_quarrelsome             4211 non-null   category
 5   pers_dependable_self-disciplined      4211 non-null   category
 6   pers_anxious_easily_upset             4211 non-null   category
 7   pers_open_to_new_experiences_complex  4211 non-null   category
 8   pers_sympathetic_warm                 4211 non-null   category
 9   pers_disorganized_careless            4211 non-null   category
 10  pers_reserved_quiet                   4211 non-null   category
 11  pers

In [117]:
# Little bit worried about the encoding of qualitative responses, since there's so many languages and unique characters
print(data.loc[:,'qual_reasons'])

# Should bear this in mind for future

0                     Je suis prête à m’engager, organise
1       Jeune diplôme je suis prête à entrer dans le m...
2       La nouvelle technologie, les nouveaux moyens m...
3       Je suis prete à passer a une nouvelle étape de...
4       Sa fait 4ans  que je suis en poste de chef d'é...
                              ...                        
4206    I am willing and ready for the next set of que...
4207                                         Cody Reasons
4208                                      Italian reasons
4209                                           El Reasons
4210                                      Reasons Spanish
Name: qual_reasons, Length: 4211, dtype: object


In [118]:
# Split batch group column between country and sector by space delimiter
data[['country', 'sector']] = data['batch'].str.split(' ', n=1, expand=True)

# Drop the original batch column
data.drop(columns=['batch'], inplace=True)

# set the new columns as first two columns
data = data[['country', 'sector'] + [col for col in data.columns if col not in ['country', 'sector']]]

# remove index column if it exists
data.head()

,country,sector,gender,age,pers_extraverted_enthusiastic,pers_critical_quarrelsome,pers_dependable_self-disciplined,pers_anxious_easily_upset,pers_open_to_new_experiences_complex,pers_sympathetic_warm,...,wfs_fcollea,wfs_fenjhyb,wfs_fexcess_wk,wfs_ffin_fair,wfs_flearn_dev,wfs_fmean_full,wfs_fpoorman,wfs_ftoxic,wfs_fuse_skills,wfs_fwellcomp
0,FR,Tech,Female,18-24,5,1,7,3,7,7,...,15.555555,65.555557,25.555555,44.444443,86.666664,63.333332,20.000000,97.777779,91.666664,100.000000
1,FR,Tech,Female,18-24,5,2,6,3,7,7,...,48.888889,37.777779,57.777779,65.555557,76.666664,77.777779,92.222221,67.777779,93.333336,16.666666
2,FR,Tech,Female,18-24,2,3,1,4,2,2,...,86.666664,60.000000,56.666668,48.888889,76.666664,91.666664,30.000000,38.888889,72.222221,83.333336
3,FR,Tech,Female,25-40,4,6,1,3,4,1,...,25.555555,62.222221,40.000000,27.777779,96.666664,21.111111,70.000000,20.000000,93.333336,22.777779
4,FR,Tech,Female,25-40,7,5,7,5,7,7,...,91.111115,0.000000,93.333336,90.000000,84.444443,70.000000,75.555557,98.888885,83.333336,58.888889


Now we've got our data cleaned and formatted.

In [119]:
data.head()

,country,sector,gender,age,pers_extraverted_enthusiastic,pers_critical_quarrelsome,pers_dependable_self-disciplined,pers_anxious_easily_upset,pers_open_to_new_experiences_complex,pers_sympathetic_warm,...,wfs_fcollea,wfs_fenjhyb,wfs_fexcess_wk,wfs_ffin_fair,wfs_flearn_dev,wfs_fmean_full,wfs_fpoorman,wfs_ftoxic,wfs_fuse_skills,wfs_fwellcomp
0,FR,Tech,Female,18-24,5,1,7,3,7,7,...,15.555555,65.555557,25.555555,44.444443,86.666664,63.333332,20.000000,97.777779,91.666664,100.000000
1,FR,Tech,Female,18-24,5,2,6,3,7,7,...,48.888889,37.777779,57.777779,65.555557,76.666664,77.777779,92.222221,67.777779,93.333336,16.666666
2,FR,Tech,Female,18-24,2,3,1,4,2,2,...,86.666664,60.000000,56.666668,48.888889,76.666664,91.666664,30.000000,38.888889,72.222221,83.333336
3,FR,Tech,Female,25-40,4,6,1,3,4,1,...,25.555555,62.222221,40.000000,27.777779,96.666664,21.111111,70.000000,20.000000,93.333336,22.777779
4,FR,Tech,Female,25-40,7,5,7,5,7,7,...,91.111115,0.000000,93.333336,90.000000,84.444443,70.000000,75.555557,98.888885,83.333336,58.888889


Let's also check for missing values and any unique encoding schemes

In [120]:
# Check missing values and recode any non-response indicators as NA

# Check for common missing value indicators
missing_indicators = ['', ' ', 'NA', 'N/A', 'null', 'NULL', 'None', 'NONE', 
                     'missing', 'Missing', 'MISSING', '?', '-', '--', 
                     '999', '-999', '9999', '-9999', '99', '-99']

print("=== CHECKING FOR MISSING VALUE INDICATORS ===\n")

# Check each column for potential missing indicators
for col in data.columns:
    print(f"{col}:")
    
    # Get unique values (first 10 and last 10 if many)
    unique_vals = data[col].unique()
    
    if len(unique_vals) <= 20:
        suspicious = [val for val in unique_vals if str(val) in missing_indicators]
        if suspicious:
            print(f"  SUSPICIOUS: {suspicious}")
        else:
            print(f"  OK")
    else:
        # For columns with many unique values, just check if any missing indicators present
        suspicious = [val for val in unique_vals if str(val) in missing_indicators]
        if suspicious:
            print(f"  SUSPICIOUS: {suspicious}")
        else:
            print(f"  OK ({len(unique_vals)} unique values)")
    
    # Also check for unusual patterns in numeric columns
    if data[col].dtype in ['int64', 'float64', 'int8', 'float32']:
        numeric_vals = pd.to_numeric(data[col], errors='coerce')
        
        # Check for extreme values that might be missing indicators
        if not numeric_vals.isna().all():
            min_val, max_val = numeric_vals.min(), numeric_vals.max()
            
            # Flag if we see common missing codes
            extreme_vals = numeric_vals[numeric_vals.isin([99, -99, 999, -999, 9999, -9999])]
            if len(extreme_vals) > 0:
                print(f"  EXTREME VALUES: {extreme_vals.unique()}")

print("\n=== MISSING DATA SUMMARY ===")
print(data.isnull().sum()[data.isnull().sum() > 0])


=== CHECKING FOR MISSING VALUE INDICATORS ===

country:
  OK
sector:
  OK
gender:
  OK
age:
  OK
pers_extraverted_enthusiastic:
  OK
pers_critical_quarrelsome:
  OK
pers_dependable_self-disciplined:
  OK
pers_anxious_easily_upset:
  OK
pers_open_to_new_experiences_complex:
  OK
pers_sympathetic_warm:
  OK
pers_disorganized_careless:
  OK
pers_reserved_quiet:
  OK
pers_calm_emotionally_stable:
  OK
pers_conventional_uncreative:
  OK
prep_level:
  OK (88 unique values)
qual_reasons:
  SUSPICIOUS: ['-']
el_lbal_anx:
  OK
el_lbal_sat:
  OK
el_lcnt_com:
  OK
el_lcnt_psy:
  OK
el_lcon_inc:
  OK
el_lcon_soc:
  OK
el_lcont_con:
  OK
el_lcont_imp:
  OK
el_lgen:
  OK
el_ljou_opt:
  OK
el_ljou_pro:
  OK
el_lres_fin:
  OK
el_lres_ski:
  OK
el_ltra_aut:
  OK
el_ltra_des:
  OK
wfl_lcarprom_d:
  OK
wfl_lcollea:
  OK
wfl_lenjhyb:
  OK
wfl_lexcess_wk:
  OK
wfl_lfin_lair:
  OK
wfl_llearn_dev:
  OK
wfl_lmean_full:
  OK
wfl_lpoorman:
  OK
wfl_ltoxic:
  OK
wfl_luse_skills:
  OK
wfl_lwellcomp:
  OK
es_fbal_

Finally let's save our data for further analysis. To preserve our data types, let's save as a pickle file, rather than csv. 
Also let's add an md file as our codebook.

In [121]:
# save as pickle file
data.to_pickle("../data/survey_data_cleaned.pkl")

### Ongoing Questions

- Not understanding the fast choice responses? Why are some negative? What does it substantively represent? Not understanding the ranges at the moment
- Fast choice are implicit scores? Whereas likert are explicit scores? What does this signify?